<a href="https://colab.research.google.com/github/dinaldoap/jit-sdp-nn/blob/master/notebook/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import random
from scipy.stats import mstats

In [70]:
df = pd.read_csv('https://raw.githubusercontent.com/dinaldoap/jit-sdp-data/master/jenkins.csv')
df.head()

,fix,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,author_date_unix_timestamp,classification,contains_bug
0,False,7.0,7.0,7.0,2.641604,9.0,9.0,426.428571,100.0,0.000093,1.0,5171.0,30.227271,1472.714286,1555326371,None,False
1,False,7.0,7.0,7.0,2.750000,8.0,8.0,426.428571,100.0,6.314775,2.0,5170.0,29.227271,1471.714286,1555326363,None,False
2,False,1.0,1.0,2.0,0.906580,15.0,44.0,96.000000,4.0,0.034722,2.0,629.0,14.828373,414.000000,1554971763,None,False
3,False,1.0,1.0,1.0,0.000000,0.0,0.0,40.000000,1.0,0.000012,1.0,4.0,3.058824,3.000000,1554969774,None,False
4,False,1.0,2.0,4.0,1.662506,14.0,10.0,67.000000,6.0,21.280683,4.0,3.0,2.058824,2.000000,1554967752,Feature Addition,False


In [71]:
#df = df.sample(frac=1)
label_col = 'contains_bug'
features_cols = ['fix', 'ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age', 'nuc', 'exp', 'rexp', 'sexp', 'classification']
x = df[features_cols]
x['fix'] = x['fix'].astype('int')
df_classification = pd.get_dummies(x, columns=['classification'])
x = pd.concat([x, df_classification], axis='columns')
x = x.drop(['classification'], axis='columns')
x = x.values
y = df[label_col]
y = y.astype('category')
y = y.cat.codes
y = y.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [72]:
print(x[:3])
print(y[:3])

[[0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.64160417e+00 9.00000000e+00 9.00000000e+00 4.26428571e+02
  1.00000000e+02 9.25925926e-05 1.00000000e+00 5.17100000e+03
  3.02272705e+01 1.47271429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.64160417e+00 9.00000000e+00
  9.00000000e+00 4.26428571e+02 1.00000000e+02 9.25925926e-05
  1.00000000e+00 5.17100000e+03 3.02272705e+01 1.47271429e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.75000000e+00 8.00000000e+00 8.00000000e+00 4.26428571e+02
  1.00000000e+02 6.31477513e+00 2.00000000e+00 5.17000000e+03
  2.92272705e+01 1.47171429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.75000000e+00 8.00000000e+00
  8.00000000e+00 4.26428571e+02 1.00000000e+02 6.31477513e+00
  2.00000000e+00 5.17000000e+03 2.92272705e+01 1.47171429e+03
  0.00000000e+00 0.00000000e+00 0.000

In [0]:
val_index = int( len(x) * 0.8 )
test_index = int( len(x) * 0.9 )
x_train, y_train = x[:val_index], y[:val_index]
x_val, y_val = x[val_index:test_index], y[val_index:test_index]
x_test, y_test = x[test_index:], y[test_index:]

In [74]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.64160417e+00 9.00000000e+00 9.00000000e+00 4.26428571e+02
  1.00000000e+02 9.25925926e-05 1.00000000e+00 5.17100000e+03
  3.02272705e+01 1.47271429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.64160417e+00 9.00000000e+00
  9.00000000e+00 4.26428571e+02 1.00000000e+02 9.25925926e-05
  1.00000000e+00 5.17100000e+03 3.02272705e+01 1.47271429e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.75000000e+00 8.00000000e+00 8.00000000e+00 4.26428571e+02
  1.00000000e+02 6.31477513e+00 2.00000000e+00 5.17000000e+03
  2.92272705e+01 1.47171429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.75000000e+00 8.00000000e+00
  8.00000000e+00 4.26428571e+02 1.00000000e+02 6.31477513e+00
  2.00000000e+00 5.17000000e+03 2.92272705e+01 1.47171429e+03
  0.00000000e+00 0.00000000e+00 0.000

In [0]:
mean_train = x_train.mean(axis=0)
std_train = x_train.std(axis=0)

In [76]:
print(mean_train)
print(std_train)

[2.90871477e-01 1.61432635e+00 2.60858547e+00 4.18908715e+00
 7.43390526e-01 5.00800621e+01 2.96331523e+01 8.11968929e+02
 2.32963538e+01 9.76994760e+01 2.02296354e+00 1.73095790e+03
 6.53773667e+01 8.28714959e+02 2.90871477e-01 1.61432635e+00
 2.60858547e+00 4.18908715e+00 7.43390526e-01 5.00800621e+01
 2.96331523e+01 8.11968929e+02 2.32963538e+01 9.76994760e+01
 2.02296354e+00 1.73095790e+03 6.53773667e+01 8.28714959e+02
 2.90871477e-01 1.47142488e-01 3.46521852e-02 4.71373157e-01
 1.51538660e-02 4.08068270e-02]
[4.54164354e-01 1.62736223e+00 6.93052365e+00 2.56616350e+01
 1.13236779e+00 5.86250473e+02 6.19852440e+02 1.15236538e+03
 2.84287684e+01 2.37802272e+02 7.67653474e+00 2.11471132e+03
 6.96820390e+01 1.34456679e+03 4.54164354e-01 1.62736223e+00
 6.93052365e+00 2.56616350e+01 1.13236779e+00 5.86250473e+02
 6.19852440e+02 1.15236538e+03 2.84287684e+01 2.37802272e+02
 7.67653474e+00 2.11471132e+03 6.96820390e+01 1.34456679e+03
 4.54164354e-01 3.54247902e-01 1.82897270e-01 4.99179

In [0]:
x_train = (x_train - mean_train) / std_train
x_val = (x_val - mean_train) / std_train
x_test = (x_test - mean_train) / std_train

In [78]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[-6.40454219e-01  3.30944983e+00  6.33633872e-01  1.09537559e-01
   1.67632253e+00 -7.00725440e-02 -3.32872003e-02 -3.34564335e-01
   2.69809951e+00 -4.10842935e-01 -1.33258504e-01  1.62671948e+00
  -5.04435529e-01  4.78964178e-01 -6.40454219e-01  3.30944983e+00
   6.33633872e-01  1.09537559e-01  1.67632253e+00 -7.00725440e-02
  -3.32872003e-02 -3.34564335e-01  2.69809951e+00 -4.10842935e-01
  -1.33258504e-01  1.62671948e+00 -5.04435529e-01  4.78964178e-01
  -6.40454219e-01 -4.15365869e-01 -1.89462561e-01  1.05899079e+00
  -1.24044505e-01 -2.06259221e-01]
 [-6.40454219e-01  3.30944983e+00  6.33633872e-01  1.09537559e-01
   1.77204746e+00 -7.17782996e-02 -3.49004875e-02 -3.34564335e-01
   2.69809951e+00 -3.84288594e-01 -2.99139370e-03  1.62624660e+00
  -5.18786429e-01  4.78220444e-01 -6.40454219e-01  3.30944983e+00
   6.33633872e-01  1.09537559e-01  1.77204746e+00 -7.17782996e-02
  -3.49004875e-02 -3.34564335e-01  2.69809951e+00 -3.84288594e-01
  -2.99139370e-03  1.62624660e+00 -5.1878

In [0]:
def create_sampler(y):
  _, counts = np.unique(y, return_counts=True)
  n_samples = len(y)
  class_weights = n_samples / counts
  weights = class_weights[y]
  fade = reversed(range(n_samples))
  fading_factor = 1 #0.999999
  fade = [fading_factor**x for x in (fade)]
  weights = weights * fade
  return data.WeightedRandomSampler(weights=weights, num_samples=n_samples, replacement=True)

In [0]:
sampler = create_sampler(y_train)

In [0]:
x_train, y_train = torch.from_numpy(x_train), torch.from_numpy(y_train)
x_val, y_val = torch.from_numpy(x_val), torch.from_numpy(y_val)
x_test, y_test = torch.from_numpy(x_test), torch.from_numpy(y_test)

In [0]:
train_dataset = data.TensorDataset(x_train, y_train)
val_dataset = data.TensorDataset(x_val, y_val)
test_dataset = data.TensorDataset(x_test, y_test)

train_dataloader = data.DataLoader(train_dataset, batch_size=512, sampler=sampler)
val_dataloader = data.DataLoader(val_dataset, batch_size=32)
test_dataloader = data.DataLoader(test_dataset, batch_size=32)

In [83]:
inputs, targets = next(iter(train_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(val_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(test_dataloader))
print(inputs[:3])
print(targets[:3])

tensor([[ 1.5614, -0.3775,  0.2008,  0.1095,  1.5703, -0.0360,  0.0054,  0.5914,
          2.1353, -0.0578,  0.5181, -0.5745, -0.7845, -0.3813,  1.5614, -0.3775,
          0.2008,  0.1095,  1.5703, -0.0360,  0.0054,  0.5914,  2.1353, -0.0578,
          0.5181, -0.5745, -0.7845, -0.3813,  1.5614, -0.4154, -0.1895, -0.9443,
         -0.1240, -0.2063],
        [-0.6405, -0.3775,  0.9222,  2.2918,  4.4708,  0.2216,  0.2426, -0.6804,
         -0.2918,  1.2221,  1.6905, -0.6828,  0.3564, -0.4854, -0.6405, -0.3775,
          0.9222,  2.2918,  4.4708,  0.2216,  0.2426, -0.6804, -0.2918,  1.2221,
          1.6905, -0.6828,  0.3564, -0.4854, -0.6405, -0.4154, -0.1895,  1.0590,
         -0.1240, -0.2063],
        [ 1.5614,  0.2370,  0.0565, -0.0463,  0.3892, -0.0462, -0.0430,  0.6694,
          0.6227, -0.3754,  0.1273, -0.2199,  1.2378, -0.1798,  1.5614,  0.2370,
          0.0565, -0.0463,  0.3892, -0.0462, -0.0430,  0.6694,  0.6227, -0.3754,
          0.1273, -0.2199,  1.2378, -0.1798,  1.5614,

In [0]:
class Classifier(nn.Module):
  def __init__(self, input_size, hidden_size, drop_prob, epoch=None, val_gmean=None):
    super(Classifier, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.drop_prob = drop_prob
    self.epoch = epoch
    self.val_gmean = val_gmean
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fcout = nn.Linear(hidden_size, 1)
    self.dropout = nn.Dropout(drop_prob)
  
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = self.dropout(x)
    x = torch.sigmoid(self.fcout(x))
    return x

In [85]:
classifier = Classifier(input_size=x.shape[1], hidden_size=x.shape[1], drop_prob=0.5)
classifier

Classifier(
  (fc1): Linear(in_features=34, out_features=34, bias=True)
  (fcout): Linear(in_features=34, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [0]:
filename = 'classifier.cpt'
def save_classifier(classifier):
  checkpoint = {
      'input_size': classifier.input_size,
      'hidden_size': classifier.hidden_size,
      'drop_prob': classifier.drop_prob,
      'val_gmean': classifier.val_gmean,
      'epoch': classifier.epoch,
      'state_dict': classifier.state_dict()
  }
  with open(filename, 'wb') as f:
    torch.save(checkpoint, f)

def load_classifier():
  with open(filename, 'rb') as f:
    checkpoint = torch.load(f)
  classifier = Classifier(checkpoint['input_size'], checkpoint['hidden_size'], checkpoint['drop_prob'], checkpoint['epoch'], checkpoint['val_gmean'])
  classifier.load_state_dict(checkpoint['state_dict'])
  return classifier

In [0]:
save_classifier(classifier)
classifier = load_classifier()

In [88]:
if torch.cuda.is_available():
  classifier = classifier.cuda()
  inputs = inputs.cuda()

classifier(inputs[:3].float())

tensor([[0.5375],
        [0.6136],
        [0.5338]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
criterion = nn.BCELoss()
optimizer = optim.Adam(params=classifier.parameters(), lr=0.003)

In [0]:
def calc_loss(classifier, dataloader):
  with torch.no_grad():
    classifier.eval()
    loss = 0
    for inputs, targets in dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

      outputs = classifier(inputs.float())
      batch_loss = criterion(outputs.squeeze(), targets.float())
      loss += batch_loss.item()

    return loss / len(dataloader)

def calc_recalls(targets, predictions):
  n_classes = 2
  confusion_matrix, _, _ = np.histogram2d(targets.detach().cpu().numpy(), predictions.squeeze().detach().cpu().numpy(), bins=n_classes)
  recalls = np.diag(confusion_matrix) / ((np.sum(confusion_matrix, axis=1)) + 1e-12)
  return recalls

def calc_gmean(classifier, dataloader):
  with torch.no_grad():
    classifier.eval()
    recalls = np.zeros((2))    
    for inputs, targets in dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()
      
      outputs = classifier(inputs.float())
      predictions = torch.round(outputs).int()
      recalls += calc_recalls(targets, predictions)
    recalls = recalls / len(dataloader)
    return mstats.gmean(recalls)    

In [95]:
train_loss = 0
epochs = 200
for epoch in range(epochs):
  classifier.train()
  for inputs, targets in train_dataloader:
    if torch.cuda.is_available():
      inputs, targets = inputs.cuda(), targets.cuda()

    outputs = classifier(inputs.float())
    loss = criterion(outputs.squeeze(), targets.float())
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss = train_loss / len(train_dataloader)
  val_loss = calc_loss(classifier, val_dataloader)
  val_gmean = calc_gmean(classifier, val_dataloader)
  print('Epoch: {}, Train loss: {}, Val loss: {}, Val g-mean: {}'.format(epoch, train_loss, val_loss, val_gmean))

  if classifier.val_gmean is None or val_gmean > classifier.val_gmean:
    classifier.epoch = epoch
    classifier.val_gmean = val_gmean
    save_classifier(classifier)

classifier.epoch = epoch
classifier.val_gmean = val_gmean

Epoch: 0, Train loss: 0.5020032493691695, Val loss: 0.4594741722470836, Val g-mean: 0.7482994546651928
Epoch: 1, Train loss: 0.5183972825303962, Val loss: 0.4690330032455294, Val g-mean: 0.7476285293160934
Epoch: 2, Train loss: 0.5174984676003855, Val loss: 0.4513895974347466, Val g-mean: 0.7454338174499954
Epoch: 3, Train loss: 0.520419270695198, Val loss: 0.4668129488433662, Val g-mean: 0.7475068763372864
Epoch: 4, Train loss: 0.5163343086289981, Val loss: 0.4630037667719941, Val g-mean: 0.7595038189193808
Epoch: 5, Train loss: 0.5200797581736655, Val loss: 0.5095403486568677, Val g-mean: 0.7400057963637188
Epoch: 6, Train loss: 0.5167559894136807, Val loss: 0.45896482036301967, Val g-mean: 0.7512806666757215
Epoch: 7, Train loss: 0.5176199429187133, Val loss: 0.5084556813694929, Val g-mean: 0.7427583984555794
Epoch: 8, Train loss: 0.5120209245121086, Val loss: 0.5047286121468795, Val g-mean: 0.750406097204685
Epoch: 9, Train loss: 0.5140449901114316, Val loss: 0.5043431967496872, Va

In [0]:
def evaluate(classifier):
  if torch.cuda.is_available():
    classifier = classifier.cuda()
  train_gmean = gmean(classifier, train_dataloader)
  test_gmean = gmean(classifier, test_dataloader)
  print('Epoch: {}'.format(classifier.epoch))
  print('Train g-mean: {}'.format(train_gmean.item()))
  print('Val g-mean: {}'.format(classifier.val_gmean))
  print('Test g-mean: {}'.format(test_gmean.item()))

In [100]:
print('Best classifier')
evaluate(load_classifier())

Best classifier
Epoch: 157
Train g-mean: 0.7662956504243538
Val g-mean: 0.7691076689455674
Test g-mean: 0.7367720310994109


In [101]:
print('Last classifier')
evaluate(classifier)

Last classifier
Epoch: 199
Train g-mean: 0.773016252273258
Val g-mean: 0.7568062728394659
Test g-mean: 0.7241882423046799
